In [1]:
import os
import sys

current_dir = os.getcwd()

parent_dir = os.path.abspath(os.path.join(current_dir, ".."))

if parent_dir not in sys.path:
    sys.path.append(parent_dir)

In [2]:
from treemort.utils.config import setup

config_file_path = "../configs/domain_adversarial_unet_bs8_cs256.txt"
conf = setup(config_file_path)

# Modified Config Variables for Local Execution; comment on HPC
conf.data_folder_finnish = "/Users/anisr/Documents/dead_trees/Finland/RGBNIR/25cm"
conf.data_folder_us = "/Users/anisr/Documents/dead_trees/USA_MO/RGBNIR/60cm"
conf.output_dir = os.path.join("..", conf.output_dir)

print(conf)

2024-10-14 22:37:06,098 - INFO - Using config file: ../configs/domain_adversarial_unet_bs8_cs256.txt
2024-10-14 22:37:06,104 - INFO - Configuration successfully loaded.


Namespace(data_folder_finnish='/Users/anisr/Documents/dead_trees/Finland/RGBNIR/25cm', data_folder_us='/Users/anisr/Documents/dead_trees/USA_MO/RGBNIR/60cm', hdf5_file_finnish='Finland_RGBNIR_25cm.h5', hdf5_file_us='USA_MO_RGBNIR_60cm.h5', model='flair_unet', backbone=None, model_weights='best', learning_rate=0.0001, activation='sigmoid', loss='hybrid', train_crop_size=256, val_crop_size=256, test_crop_size=256, input_channels=4, output_channels=1, class_weights=[0.1, 0.9], epochs=100, train_batch_size=8, val_batch_size=8, test_batch_size=8, val_size=0.2, test_size=0.1, resume=False, lambda_adv=0.5, domain_loss='cross_entropy', output_dir='../output/flair_unet')


In [3]:
from treemort.main import run

eval_only = False

run(conf, eval_only)

/Users/anisr/Documents/TreeSeg/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


AssertionError: [ERROR] Finnish data file Finland_RGBNIR_25cm.h5 does not exist.